## Module Paper
## Seminar Computing Meaning

## Importing Required Libraries

In [1]:
### Importing Required Libraries

from nltk.corpus import wordnet 
from nltk import word_tokenize
from bs4 import BeautifulSoup
import sys
import os
import random
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from scipy import spatial
import random
import numpy as np
from nltk.corpus import stopwords
from nltk.corpus import brown
import multiprocessing
from gensim.models.fasttext import FastText
from sentence_transformers import SentenceTransformer, models
from torch import nn

## Reading Data

### 1. Download all the relevant data from SemCor, Senseval-2 and Senseval-31.Make a random selection of 5000 sentences from the SemCor data as described in the paper. Download the pre-trained word embeddings and sense embeddings from AutoExtend2 (Rothe and Schütze, 2015).

### arr_sem_cor_file contains all file names in semcor1.7.1. The roordir indicates the physical location where it is stored in the computer. semcor1.7.1 has 352 manually annotated documents with information regarding lemma, POS-Tag and wnsn number.

In [2]:
rootdir = 'path\\semcor1.7.1'

arr_sem_cor_file=[]
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if 'br' in file:
            var1=str(os.path.join(subdir, file))
            arr_sem_cor_file.append(var1)


In [225]:
print("Number of Documents in SemCor",len(arr_sem_cor_file))

Number of Documents in SemCor 352


### arr_senseval2_file contains all file names in senseval2.semcor

In [4]:
rootdir = 'path\\senseval2.semcor'

arr_senseval2_file=[]
for subdir, dirs, files in os.walk(rootdir):
    if subdir==r"path\senseval2.semcor\wordnet1.7.1":
        for file in files:
            var1=str(os.path.join(subdir, file))
            arr_senseval2_file.append(var1)

In [226]:
print("Number of Documents in SenseEval2",len(arr_senseval2_file))

Number of Documents in SenseEval2 3


### arr_senseval3_file contains all file names in senseval3.semcor

In [6]:
rootdir = 'path\\senseval3.semcor'

arr_senseval3_file=[]
for subdir, dirs, files in os.walk(rootdir):
    if subdir==r"path\senseval3.semcor\wordnet1.7.1":
        for file in files:
            var1=str(os.path.join(subdir, file))
            arr_senseval3_file.append(var1)

In [227]:
print("Number of Documents in SenseEval3",len(arr_senseval3_file))

Number of Documents in SenseEval3 3


### Perform Pos tag mapping between Input Embedding and lexeme embedding. Here n indicates all forms of Noun, v is Verb, a is Adjective and r is Adverb.And in further places we see s is Adjective. This is Wordnet to SemCor Pos-Tag mapping.

In [8]:
dict_for_pos_tags={"NN":"n","VB":"v","RB":"r","JJ":"a","NNP":"n","NNPS":"n","NP":"n","NPS":"n",
                   "NNS":"n","JJR":"a","JJS":"a","RBR":"r",
                   "RBS":"r","VBD":"v","VBG":"v","VBN":"v","VBP":"v","MD":"v",
                   "VBZ":"v","WRB":"r"} 

### Dictionaries are created from storing sentences from SemCor, SenseEval2 and SenseEval3.

In [9]:
dict2={} #Used for storing Semcor

In [10]:
dict_senseval2={} #Used for storing senseval2

In [11]:
dict_senseval3={} #Used for storing senseval3

### Semcor data reading and creating a dictionary. From this 5000 random sentences will be selected

In [12]:
## Semcor data reading and creating a dictionary
counter=0
for file in arr_sem_cor_file:
    with open(file) as fp:
        soup = BeautifulSoup(fp, "html.parser")

    tags = soup.find_all("s")
    for i in tags:
        tag=i.find_all("wf", {"cmd":"done"})
        sen1=""
        dict3={}
        for j in tag:
            if type(j.get("lemma"))==str:
                flag=True
                if (not (j.get("ot"))):
                    sen1+=j.text
                    sen1+=" "
                    key=j.text
                    if j.get("pos") in dict_for_pos_tags.keys():
                        pos=dict_for_pos_tags[j.get("pos")]
                    else:
                        print("Pos not available")
                    value=j.get("lemma")+"-"+pos+"-"+j.get("wnsn")
                    dict3[key]=value
            else:
                flag=False
        
        if flag:
            key=counter
            value=sen1
            dict2[key]=[value,dict3]
            counter+=1
            
print(dict2)  

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Senseval2 data reading and creating a dictionary. Entire Corpus is taken for evaluation. In the Output section we can see the nested dictionary structure.

In [14]:
## Senseval2 data reading and creating a dictionary
counter=0
for file in arr_senseval2_file:
    with open(file) as fp:
        soup = BeautifulSoup(fp, "html.parser")

    tags = soup.find_all("s")
    for i in tags:
        tag=i.find_all("wf", {"cmd":"done"})
        sen1=""
        dict3={}
        for j in tag:
            if type(j.get("lemma"))==str:
                flag=True
                if (not (j.get("ot"))):
                    sen1+=j.text
                    sen1+=" "
                    key=j.text
                    
                    if j.get("pos") in dict_for_pos_tags.keys():
                        pos=dict_for_pos_tags[j.get("pos")]
                        value=j.get("lemma")+"-"+pos+"-"+j.get("wnsn")
                    else:
                        print(j.get("pos"))
                        print("Pos not available")
                    dict3[key]=value
            else:
                flag=False
        
        if flag:
            key=counter
            value=sen1
            dict_senseval2[key]=[value,dict3]
            counter+=1
            
print(dict_senseval2)  

{0: ['art change-ringing is peculiar English most English peculiarities unintelligible rest world ', {'art': 'art-n-3', 'change-ringing': 'change_ringing-n-1', 'is': 'be-v-1', 'peculiar': 'peculiar-a-2;4', 'English': 'english-a-1', 'most': 'most-a-1', 'peculiarities': 'peculiarity-n-1;2', 'unintelligible': 'unintelligible-a-2', 'rest': 'rest-n-1', 'world': 'world-n-8'}], 1: ['Tailors England scenes evoke rural England loveliest ancient stone church stands fields sound bells cascading tower calling faithful evensong ', {'Tailors': 'tailor-n-1', 'England': 'england-n-1', 'scenes': 'scene-n-3', 'evoke': 'evoke-v-5', 'rural': 'rural-a-1', 'loveliest': 'lovely-a-1', 'ancient': 'ancient-a-2', 'stone': 'stone-n-3', 'church': 'church-n-2', 'stands': 'stand-v-3', 'fields': 'field-n-9', 'sound': 'sound-n-1', 'bells': 'bell-n-1', 'cascading': 'cascade-v-1', 'tower': 'tower-n-1', 'calling': 'call-v-4', 'faithful': 'faithful-n-2', 'evensong': 'evensong-n-2'}], 2: ['parishioners stop chat church doo

### Senseval3 data reading and creating a dictionary. Entire Corpus is taken for evaluation

In [16]:
## Senseval3 data reading and creating a dictionary
counter=0
for file in arr_senseval3_file:
    with open(file) as fp:
        soup = BeautifulSoup(fp, "html.parser")

    tags = soup.find_all("s")
    for i in tags:
        tag=i.find_all("wf", {"cmd":"done"})
        sen1=""
        dict3={}
        for j in tag:
            if type(j.get("lemma"))==str:
                flag=True
                if (not (j.get("ot"))):
                    sen1+=j.text
                    sen1+=" "
                    key=j.text
                    if j.get("pos") in dict_for_pos_tags.keys():
                        pos=dict_for_pos_tags[j.get("pos")]
                    else:
                        print("Pos not available")
                    value=j.get("lemma")+"-"+pos+"-"+j.get("wnsn")
                    dict3[key]=value
            else:
                flag=False
        
        if flag:
            key=counter
            value=sen1
            dict_senseval3[key]=[value,dict3]
            counter+=1
            
print(dict_senseval3)  

{0: ["'s man said ", {"'s": 'be-v-1', 'man': 'man-n-1', 'said': 'say-v-2'}], 1: ['peered drinking companion bleary tear-filled eyes ', {'peered': 'peer-v-1', 'drinking': 'drinking-n--1', 'companion': 'companion-n-1', 'bleary': 'bleary-a-2', 'tear-filled': 'UNKNOWN-a--1', 'eyes': 'eye-n-1'}], 2: ['had ready answer much surprise fit coughing ', {'had': 'have-v-1', 'ready': 'ready-a-2', 'answer': 'answer-n-5', 'much': 'much-a-1', 'surprise': 'surprise-n-1', 'fit': 'fit-n-2', 'coughing': 'coughing-n-1'}], 3: ['Was man drunk crazy new-found buddy matched drink drink lost count man eyes were clear ', {'Was': 'be-v-1', 'man': 'man-n-1', 'drunk': 'drunk-a-1', 'crazy': 'crazy-a-1', 'new-found': 'newfound-a-1', 'buddy': 'buddy-n-1', 'matched': 'match-v-1', 'drink': 'drink-n-1', 'lost': 'lose-v-1', 'count': 'count-n-1', 'eyes': 'eye-n-1', 'were': 'be-v-1', 'clear': 'clear-a-4'}], 4: ['guy is rocker thought looked_away eyes ', {'guy': 'guy-n-1', 'is': 'be-v-1', 'rocker': 'rocker-n-4', 'thought': '

## Simple Lesk

### 2. Implement two baseline methods: most common sense and the Plain Lesk algorithm.

### Define Function Words - Function words do not provide much information and therefore are removed from the gloss vector.

In [47]:
functionwords = ['about', 'across', 'against', 'along', 'around', 'at',
                 'behind', 'beside', 'besides', 'by', 'despite', 'down',
                 'during', 'for', 'from', 'in', 'inside', 'into', 'near', 'of',
                 'off', 'on', 'onto', 'over', 'through', 'to', 'toward',
                 'with', 'within', 'without', 'anything', 'everything',
                 'anyone', 'everyone', 'ones', 'such', 'it', 'itself',
                 'something', 'nothing', 'someone', 'the', 'some', 'this',
                 'that', 'every', 'all', 'both', 'one', 'first', 'other',
                 'next', 'many', 'much', 'more', 'most', 'several', 'no', 'a',
                 'an', 'any', 'each', 'no', 'half', 'twice', 'two', 'second',
                 'another', 'last', 'few', 'little', 'less', 'least', 'own',
                 'and', 'but', 'after', 'when', 'as', 'because', 'if', 'what',
                 'where', 'which', 'how', 'than', 'or', 'so', 'before', 'since',
                 'while', 'although', 'though', 'who', 'whose', 'can', 'may',
                 'will', 'shall', 'could', 'be', 'do', 'have', 'might', 'would',
                 'should', 'must', 'here', 'there', 'now', 'then', 'always',
                 'never', 'sometimes', 'usually', 'often', 'therefore',
                 'however', 'besides', 'moreover', 'though', 'otherwise',
                 'else', 'instead', 'anyway', 'incidentally', 'meanwhile']

In [247]:
def check_overlapcontext( synset, sentence ):
    
    gloss = set(word_tokenize(synset.definition()))
    for i in synset.examples():
         gloss.union(i)
    gloss = gloss.difference( functionwords )
    if isinstance(sentence, str):
        sentence = set(sentence.split(" "))
    elif isinstance(sentence, list):
        sentence = set(sentence)
    elif isinstance(sentence, set):
        pass
    else:
        return
    sentence = sentence.difference( functionwords )
    return len( gloss.intersection(sentence) )

def plain_lesk( word, sentence ):
    bestsense = None
    maxoverlap_count = 0
    word=wordnet.morphy(word) if wordnet.morphy(word) is not None else word
    for sense in wordnet.synsets(word):
        overlap = check_overlapcontext(sense,sentence)
        for h in sense.hyponyms():
            overlap += check_overlapcontext( h, sentence )
        if overlap > maxoverlap_count:
                maxoverlap_count = overlap
                bestsense = sense
    return bestsense

In [248]:
def evaluate_plain_lesk(sentence, word_dict):
    arr_pl=[]
    set_of_words=set(word_tokenize(sentence))
    for i in set_of_words:
        predicted_synset = plain_lesk(i,sentence)
        if predicted_synset is not None:
            if i in word_dict.keys():
                lemma=word_dict[i].split("-")[0]
                pos=word_dict[i].split("-")[1]
                wnsn=word_dict[i].split("-")[2]
                true_synset=lemma+'.'+pos+'.0'+wnsn    
                predicted_synset=predicted_synset.name()
                acc=find_accuracy_lesk(predicted_synset,true_synset)
                arr_pl.append(acc)
    return arr_pl      

In [249]:
def find_accuracy_lesk(predicted_synset,true_synset):
    
    predicted_synset_arr=predicted_synset.split(".")
    true_synset_arr=true_synset.split(".")
    if ';' in true_synset_arr[2]:
        true_synset_arr_1=true_synset_arr[2].split(";")
        true_synset_arr_1=[i.zfill(2) for i in true_synset_arr_1]
    else:
        true_synset_arr_1=true_synset_arr[2]

    if (predicted_synset_arr[1]==true_synset_arr[1] and predicted_synset_arr[2] in true_synset_arr_1):
        return 1
    else:
        return 0

### Function to generate random numbers

In [22]:
random_list=[]
for i in range(0,5000):
    x = random.randint(1,34374)
    random_list.append(x)
    
print("Length of random numbers",len(random_list))

Length of random numbers 5000


### Evaluation on Semcor  for PlainLesk Algorithm

In [51]:
##Semcor Computation
final_arr_plainlesk=[]
for i in random_list:
    final_arr_plainlesk.append(evaluate_plain_lesk(dict2[i][0],dict2[i][1]))

flat_list = [item for sublist in final_arr_plainlesk for item in sublist]
accuracy_final_plainlesk=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy is Plain Lesk with Semcor Dataset",accuracy_final_plainlesk*100)

Length of final words 16558
Final Accuracy is Plain Lesk with Semcor Dataset 38.58557796835367


### Evaluation on Senseval2 for PlainLesk Algorithm

In [250]:
final_arr_plainlesk_senseval2=[]
for i in dict_senseval2.keys():
    final_arr_plainlesk_senseval2.append(evaluate_plain_lesk(dict_senseval2[i][0],dict_senseval2[i][1]))
    

flat_list = [item for sublist in final_arr_plainlesk_senseval2 for item in sublist]
accuracy_final_plainlesk_senseval2=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy is for Senseval2 for Plain Lesk is",accuracy_final_plainlesk_senseval2*100)

Length of final words 1410
Final Accuracy is for Senseval2 for Plain Lesk is 43.262411347517734


### Evaluation on Senseval3 for PlainLesk Algorithm

In [251]:
##Senseval3 Computation
final_arr_plainlesk_senseval3=[]
for i in dict_senseval3.keys():
    final_arr_plainlesk_senseval3.append(evaluate_plain_lesk(dict_senseval3[i][0],dict_senseval3[i][1]))

flat_list = [item for sublist in final_arr_plainlesk_senseval3 for item in sublist]
accuracy_final_plainlesk_senseval3=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy is for Plain Lesk Senseval3",accuracy_final_plainlesk_senseval3*100)

Length of final words 1252
Final Accuracy is for Plain Lesk Senseval3 37.77955271565495


### Most- Common Sense Algorithm

In [253]:
def most_common_sense(lemma):
    """returns most common sense for input word
    Arguments are 
        lemma: (string) the lemma of a word
    return: first sense for the lemma (predicted_synset)
    """
    synset =wordnet.synsets(lemma)
    if len(synset) > 0:
        return synset[0]
    else:
        return None

In [254]:
def evaluate_most_common_sense(sentence,word_dict):
    arr_msc=[]
    set_of_words=set(word_tokenize(sentence))
    for i in set_of_words:
        predicted_synset = most_common_sense(i)
        if predicted_synset is not None:
            if i in word_dict.keys():
                if (i not in word_dict.keys() and i+'.' in word_dict.keys()): 
                    i=i+'.'

                lemma=word_dict[i].split("-")[0]
                pos=word_dict[i].split("-")[1]
                wnsn=word_dict[i].split("-")[2]
                true_synset=lemma+'.'+pos+'.0'+wnsn    
                predicted_synset=predicted_synset.name()
                acc=find_accuracy(predicted_synset,true_synset)
                arr_msc.append(acc)
    return arr_msc   

In [255]:
def find_accuracy(predicted_synset,true_synset):
    
    """
    Input: Predicted Synset and the True Synset
    Output: Compares both of them. If they match, returns 1 otherwise returns 0.
    
    """
    
    predicted_synset_arr=predicted_synset.split(".")
    true_synset_arr=true_synset.split(".")
    if ';' in true_synset_arr[2]:
        true_synset_arr_1=true_synset_arr[2].split(";")
        true_synset_arr_1=[i.zfill(2) for i in true_synset_arr_1]
    else:
        true_synset_arr_1=true_synset_arr[2]
    if predicted_synset_arr[1]=='s': # s here indicates satellite adjective
        predicted_synset_arr[1]='a'
    if (predicted_synset_arr[1]==true_synset_arr[1] and predicted_synset_arr[2] in true_synset_arr_1):
        return 1
    else:
        return 0

In [233]:
##Semcor Computation
final_arr_mcs=[]
for i in random_list:
    final_arr_mcs.append(evaluate_most_common_sense(dict2[i][0],dict2[i][1]))

flat_list = [item for sublist in final_arr_mcs for item in sublist]
accuracy_final_mcs=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy for Most Common Sense with Semcor Dataset",accuracy_final_mcs*100)

Length of final words 29189
Final Accuracy for Most Common Sense with Semcor Dataset 50.0188427147213


In [256]:
mcs_senseval2=[]
for i in dict_senseval2.keys():
    mcs_senseval2.append(evaluate_most_common_sense(dict_senseval2[i][0],dict_senseval2[i][1]))
    

flat_list = [item for sublist in mcs_senseval2 for item in sublist]
accuracy_final_mcs_senseval2=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy for Most Common Sense Algorithm with Senseval2 is",accuracy_final_mcs_senseval2*100)

Length of final words 2373
Final Accuracy for Most Common Sense Algorithm with Senseval2 is 49.3889591234724


In [257]:
##Senseval3 Computation
mcs_senseval3=[]
for i in dict_senseval3.keys():
    mcs_senseval3.append(evaluate_most_common_sense(dict_senseval3[i][0],dict_senseval3[i][1]))

flat_list = [item for sublist in mcs_senseval3 for item in sublist]
accuracy_final_mcs_senseval3=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy for Most Common Sense with Senseval3 is",accuracy_final_mcs_senseval3*100)

Length of final words 1941
Final Accuracy for Most Common Sense with Senseval3 is 47.2952086553323


## Distributional Lesk Embedding

### 3. Implement the method proposed by Oele and van Noord (2017) using the pre-trained word embeddings.

### Load the Word Embeddings from Word2Vec Model using Google-News Dataset

In [18]:
goog_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [19]:
lines = open("embeddings\\lexemes.txt").read().splitlines()

### dict1 is used to store lexeme embeddings from reading the lexemes.txt from AutoExtend Embeddings.

In [20]:
dict1={}
for line in lines:
    key=line.split()[0]
    value=line.split()[1:]
    final_vec=[float(i) for i in value]
    dict1[key]=final_vec

### Define Function Words - Function words do not provide much information and therefore are removed from the gloss vector.

In [21]:
functionwords = ['about', 'across', 'against', 'along', 'around', 'at',
                 'behind', 'beside', 'besides', 'by', 'despite', 'down',
                 'during', 'for', 'from', 'in', 'inside', 'into', 'near', 'of',
                 'off', 'on', 'onto', 'over', 'through', 'to', 'toward',
                 'with', 'within', 'without', 'anything', 'everything',
                 'anyone', 'everyone', 'ones', 'such', 'it', 'itself',
                 'something', 'nothing', 'someone', 'the', 'some', 'this',
                 'that', 'every', 'all', 'both', 'one', 'first', 'other',
                 'next', 'many', 'much', 'more', 'most', 'several', 'no', 'a',
                 'an', 'any', 'each', 'no', 'half', 'twice', 'two', 'second',
                 'another', 'last', 'few', 'little', 'less', 'least', 'own',
                 'and', 'but', 'after', 'when', 'as', 'because', 'if', 'what',
                 'where', 'which', 'how', 'than', 'or', 'so', 'before', 'since',
                 'while', 'although', 'though', 'who', 'whose', 'can', 'may',
                 'will', 'shall', 'could', 'be', 'do', 'have', 'might', 'would',
                 'should', 'must', 'here', 'there', 'now', 'then', 'always',
                 'never', 'sometimes', 'usually', 'often', 'therefore',
                 'however', 'besides', 'moreover', 'though', 'otherwise',
                 'else', 'instead', 'anyway', 'incidentally', 'meanwhile',')','(','{','}','[',']',",",".",";","'","-","'s","0;"]

In [23]:
def find_gloss_embedding(sense):
    
    """
    Input: Sense of a word
    Output: Array of word embeddings of all words in the gloss.
    
    """
    
    gloss = set(word_tokenize(sense.definition()))           
    gloss = gloss.difference( functionwords )
    examples_arr=sense.examples()
    new_token_array=[word_tokenize(i) for i in examples_arr]
    examples_new_token_array = [item for sublist in new_token_array for item in sublist]
    examples_new_token_array=set(examples_new_token_array)
    gloss = gloss.union( examples_new_token_array )
    gloss = gloss.difference( functionwords )
    arr=[]
    for i in gloss:
        if i in goog_model.key_to_index:
            word_embedding=goog_model[i]
            arr.append(word_embedding)
    return arr

### calculate_meaning() is the main function which implements Distributional Lesk. It takes a sentence and a dictionary of the words and their corresponding lemma-pos-tag-wnsn and outputs the  number of correct synsets predicted.

In [203]:
def calculate_meaning(sen1,dict_for_words): 
    acc_arr=[]
    set_of_words=set(word_tokenize(sen1))  
    ## For sorting the words based on number of synsets each word has before passing it to function
    sorted_list=[]
    for i in set_of_words:
        if len(wordnet.synsets(i))>0:  
            sorted_list.append([i,len(wordnet.synsets(i))])

    sorted_list.sort(key = lambda x: x[1])
    
    dict_for_sense_embedding={}
    for word_list in sorted_list:
        word=word_list[0]
        
        final_score_arr=[]
        
            
        for sense in wordnet.synsets(word): 
            
            #Computing Gloss Vector
            arr=find_gloss_embedding(sense)
            
            gloss_embedding_sense=np.mean(arr, axis=0) #Gloss embedding for 1 sense of a word
            
            gloss_embedding_sense=gloss_embedding_sense.tolist()
            
            
            sentence1=set(word_tokenize(sen1))
            sentence1 = sentence1.difference( functionwords )


            arr2=[x for x in sentence1 if x not in word]
            ## This is for Context embedding
            arr3=[]
            for i in arr2:
                if i in dict_for_sense_embedding.keys():
                    #### This is sense embedding for disambiguated word
                    arr3=find_gloss_embedding(dict_for_sense_embedding[i])
                    
                    
                else:
                    if i in goog_model.key_to_index:  
                        word_embedding=goog_model[i]
                        arr3.append(word_embedding)

            context_embedding_sentence=np.mean(arr3, axis=0) #Context embedding for entire sentence removing the word
            context_embedding_sentence=context_embedding_sentence.tolist()
            score_first_part= spatial.distance.cosine(gloss_embedding_sense, context_embedding_sentence)

            ### Finding the offset for the particular sense
            offset=sense.offset()
            
            
            if word in dict_for_words.keys():
                lemma=dict_for_words[word].split("-")[0]
                pos=dict_for_words[word].split("-")[1]
                wnsn=dict_for_words[word].split("-")[2]
                format_word=lemma+"-wn-2.1-"+str(offset).zfill(8)+"-"+pos
                if format_word in dict1.keys():
                    lexeme_vector=dict1[format_word]
                    score_second_part= spatial.distance.cosine(lexeme_vector, context_embedding_sentence)
                    final_score=score_first_part+score_second_part
                else:
                    final_score=score_first_part
            else:
                final_score=score_first_part
            
            final_score_arr.append(final_score)
        if len(wordnet.synsets(word))!=0:
            max_score=np.argmax(final_score_arr)
            key=word
            value=wordnet.synsets(word)[max_score]
            dict_for_sense_embedding[key]=value
            
            predicted_synset_definition=wordnet.synsets(word)[max_score].definition()
            predicted_synset=wordnet.synsets(word)[max_score].name()
            
            if word in dict_for_words.keys():
                if (word not in dict_for_words.keys() and word+'.' in dict_for_words.keys()): 
                    word=word+'.'
                lemma=dict_for_words[word].split("-")[0]
                pos=dict_for_words[word].split("-")[1]
                wnsn=dict_for_words[word].split("-")[2]
                true_synset=lemma+'.'+pos+'.0'+wnsn
            
                ### Function to check if Predicted Synset and True Synset are the same
                acc=find_accuracy(predicted_synset,true_synset)
                acc_arr.append(acc)
    return acc_arr

### Function to find Accuracy between True Synset and Predicted Synset

In [204]:
def find_accuracy(predicted_synset,true_synset):
    
    """
    Input: Predicted Synset and the True Synset
    Output: Compares both of them. If they match, returns 1 otherwise returns 0.
    
    """
    
    predicted_synset_arr=predicted_synset.split(".")
    true_synset_arr=true_synset.split(".")
    if ';' in true_synset_arr[2]:
        true_synset_arr_1=true_synset_arr[2].split(";")
        true_synset_arr_1=[i.zfill(2) for i in true_synset_arr_1]
    else:
        true_synset_arr_1=true_synset_arr[2]
    if predicted_synset_arr[1]=='s': # s here indicates satellite adjective
        predicted_synset_arr[1]='a'
    if (predicted_synset_arr[1]==true_synset_arr[1] and predicted_synset_arr[2] in true_synset_arr_1):
        return 1
    else:
        return 0

## 4. Evaluate the method on the given three datasets

## Semcor Computation

In [205]:
##Semcor Computation
final_arr=[]
for i in random_list:
    final_arr.append(calculate_meaning(dict2[i][0],dict2[i][1]))

flat_list = [item for sublist in final_arr for item in sublist]
accuracy_final=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy is for Distributional Lesk for Semcor Dataset",accuracy_final*100)

Length of final words 29189
Final Accuracy is for Distributional Lesk for Semcor Dataset 42.019253828497035


## Senseval 2 Computation

In [158]:
##Senseval 2 Computation
final_arr_senseval2=[]
for i in dict_senseval2.keys():
    final_arr_senseval2.append(calculate_meaning(dict_senseval2[i][0],dict_senseval2[i][1]))

flat_list = [item for sublist in final_arr_senseval2 for item in sublist]
accuracy_final_senseval2=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy is for Senseval2 for Distributional Lesk is",accuracy_final_senseval2*100)

Length of final words 2373
Final Accuracy is for Senseval2 for Distributional Lesk is 39.94943109987358


## Senseval3 Computation

In [159]:
##Senseval3 Computation
final_arr_senseval3=[]
for i in dict_senseval3.keys():
    final_arr_senseval3.append(calculate_meaning(dict_senseval3[i][0],dict_senseval3[i][1]))

flat_list = [item for sublist in final_arr_senseval3 for item in sublist]
accuracy_final_senseval3=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy is for Senseval3 for Distributional Lesk",accuracy_final_senseval3*100)

Length of final words 1941
Final Accuracy is for Senseval3 for Distributional Lesk 39.41267387944359


## Additionally, make extensions to the work described in the paper:

### Extension 1: Experiment with removing stopwords and punctuation from the dictionary glosses, sense descriptions and contexts in the occurrences of the words before measuring the distance.


#### Here stopwords of English language are initialized

In [206]:
stopwords_list=stopwords.words("English")

### Main functionality is same as Distributional Lesk. Only Stopwords are removed from the sentence, gloss vector and context vector. They are removed from the sense definition and examples.

In [207]:
def extension_stopwords(sen1,dict_for_words): 
    acc_arr=[]
    set_of_words=set(word_tokenize(sen1))  
    set_of_words=set_of_words.difference( stopwords_list ) 
    ## For sorting the words based on number of synsets each word has before passing it to program
    sorted_list=[]
    for i in set_of_words:
        if len(wordnet.synsets(i))>0:
            sorted_list.append([i,len(wordnet.synsets(i))])

    sorted_list.sort(key = lambda x: x[1])
    
    dict_for_sense_embedding={}
    for word_list in sorted_list:
        word=word_list[0]
        
        final_score_arr=[]
        
            
        for sense in wordnet.synsets(word): 
            gloss = set(word_tokenize(sense.definition())) 
            
            gloss = gloss.difference( functionwords )
            
            ### Here is the part where union from examples is being carried out
            
            examples_arr=sense.examples()
            new_token_array=[word_tokenize(i) for i in examples_arr]
            examples_new_token_array = [item for sublist in new_token_array for item in sublist]
            examples_new_token_array=set(examples_new_token_array)
            gloss = gloss.union( examples_new_token_array )
            gloss = gloss.difference( functionwords )
            gloss = gloss.difference( stopwords_list )

            
            
            arr=[]
            for i in gloss:
                if i in goog_model.key_to_index:
                    
                    word_embedding=goog_model[i]
                    arr.append(word_embedding)
            
            
            gloss_embedding_sense=np.mean(arr, axis=0) #Gloss embedding for 1 sense of a word
            
            gloss_embedding_sense=gloss_embedding_sense.tolist()
            sentence1=set(word_tokenize(sen1))
            sentence1 = sentence1.difference( functionwords )


            arr2=[x for x in sentence1 if x not in word]
            ## This is for Context embedding
            arr3=[]
            for i in arr2:
                if i in dict_for_sense_embedding.keys():
                    #### This is sense embedding for disambiguated word
                    arr3=find_gloss_embedding(dict_for_sense_embedding[i])
                    
                else:
                    if i in goog_model.key_to_index:  
                        word_embedding=goog_model[i]
                        arr3.append(word_embedding)

            context_embedding_sentence=np.mean(arr3, axis=0) #Context embedding for entire sentence removing the word
            context_embedding_sentence=context_embedding_sentence.tolist()
            
            if (len(gloss)==0):
                score_first_part= 0
            else:
                score_first_part= spatial.distance.cosine(gloss_embedding_sense, context_embedding_sentence)

            offset=sense.offset()
            
            if word in dict_for_words.keys():
                lemma=dict_for_words[word].split("-")[0]
                pos=dict_for_words[word].split("-")[1]
                wnsn=dict_for_words[word].split("-")[2]
                format_word=lemma+"-wn-2.1-"+str(offset).zfill(8)+"-"+pos


                if format_word in dict1.keys():
                    lexeme_vector=dict1[format_word]
                    score_second_part= spatial.distance.cosine(lexeme_vector, context_embedding_sentence)
                    final_score=score_first_part+score_second_part
                else:
                    final_score=score_first_part
            else:
                final_score=score_first_part
            
            final_score_arr.append(final_score)
        if len(wordnet.synsets(word))!=0:
            max_score=np.argmax(final_score_arr)
            key=word
            value=wordnet.synsets(word)[max_score]
            dict_for_sense_embedding[key]=value
            predicted_synset_definition=wordnet.synsets(word)[max_score].definition()
            predicted_synset=wordnet.synsets(word)[max_score].name()
            
            if word in dict_for_words.keys():
                if (word not in dict_for_words.keys() and word+'.' in dict_for_words.keys()): 
                    word=word+'.'
                lemma=dict_for_words[word].split("-")[0]
                pos=dict_for_words[word].split("-")[1]
                wnsn=dict_for_words[word].split("-")[2]
                true_synset=lemma+'.'+pos+'.0'+wnsn
            
                acc=find_accuracy(predicted_synset,true_synset)
                acc_arr.append(acc)

    return acc_arr

### Evaluation on Semcor Data after Removing Stopwords

In [208]:
final_arr_semcor_wo_stopwords=[]
for i in random_list:
    final_arr_semcor_wo_stopwords.append(extension_stopwords(dict2[i][0],dict2[i][1]))

flat_list = [item for sublist in final_arr_semcor_wo_stopwords for item in sublist]
accuracy_final_wo_stopwords=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy after removing stopwords from Semcor Data is: ",accuracy_final_wo_stopwords*100)

Length of final words 25793
Final Accuracy after removing stopwords from Semcor Data is:  42.28278990423758


### Evaluation on Senseval2 after Removing Stopwords

In [161]:
##Senseval 2 Computation
final_arr_senseval2_wo_stopwords=[]
for i in dict_senseval2.keys():
    final_arr_senseval2_wo_stopwords.append(extension_stopwords(dict_senseval2[i][0],dict_senseval2[i][1]))

flat_list = [item for sublist in final_arr_senseval2_wo_stopwords for item in sublist]
accuracy_final_senseval2_wo_stopwords=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy for Senseval2 after removing stopwords is: ",accuracy_final_senseval2_wo_stopwords*100)

Length of final words 2197
Final Accuracy for Senseval2 after removing stopwords is:  39.64497041420118


### Evaluation on Senseval3 after Removing Stopwords

In [162]:
##Senseval3 Computation
final_arr_senseval3_wo_stopwords=[]
for i in dict_senseval3.keys():
    final_arr_senseval3_wo_stopwords.append(extension_stopwords(dict_senseval3[i][0],dict_senseval3[i][1]))

flat_list = [item for sublist in final_arr_senseval3_wo_stopwords for item in sublist]
accuracy_final_senseval3_wo_stopwords=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy for Senseval3 after removing stopwords is: ",accuracy_final_senseval3_wo_stopwords*100)

Length of final words 1769
Final Accuracy for Senseval3 after removing stopwords is:  39.5703787450537


### Extension 2: SemCor data come from the Brown corpus. Evaluate the results for individual categories.

In [70]:
rootdir_brown_corpus = 'path\\semcor1.7.1'

In [71]:
arr_sem_cor_file_government=[]
for subdir, dirs, files in os.walk(rootdir_brown_corpus):
    for file in files:
        if 'br-h' in file:
            var1=str(os.path.join(subdir, file))
            arr_sem_cor_file_government.append(var1)


In [72]:
arr_sem_cor_file_hobbies=[]
for subdir, dirs, files in os.walk(rootdir_brown_corpus):
    for file in files:
        if 'br-e' in file:
            var1=str(os.path.join(subdir, file))
            arr_sem_cor_file_hobbies.append(var1)

In [73]:
arr_sem_cor_file_adventure=[]
for subdir, dirs, files in os.walk(rootdir_brown_corpus):
    for file in files:
        if 'br-n' in file:
            var1=str(os.path.join(subdir, file))
            arr_sem_cor_file_adventure.append(var1)

### Creating dictionary for storing sentences for different categories of brown corpus.

In [74]:
dict_for_government={} ## For storing Government Category

In [75]:
dict_for_hobbies={} ## For storing Hobbies Category

In [76]:
dict_for_adventure={} ## For storing Adventure Category

### Semcor data reading and creating a dictionary for Brown Corpus Category Government. It contains 1489 sentences.

In [77]:
## Semcor data reading and creating a dictionary for Brown Corpus Category News

counter=0
for file in arr_sem_cor_file_government:
    with open(file) as fp:
        soup = BeautifulSoup(fp, "html.parser")

    tags = soup.find_all("s")
    for i in tags:
        tag=i.find_all("wf", {"cmd":"done"})
        sen1=""
        dict3={}
        for j in tag:
            if type(j.get("lemma"))==str:
                flag=True
                if (not (j.get("ot"))):
                    sen1+=j.text
                    sen1+=" "
                    key=j.text
                    if j.get("pos") in dict_for_pos_tags.keys():
                        pos=dict_for_pos_tags[j.get("pos")]
                    else:
                        print("Pos not available")
                    value=j.get("lemma")+"-"+pos+"-"+j.get("wnsn")
                    dict3[key]=value
            else:
                flag=False
        
        if flag:
            key=counter
            value=sen1
            dict_for_government[key]=[value,dict3]
            counter+=1
            
print(dict_for_government)  

{0: ['The_Office_of_Business_Economics U.S._Department_of_Commerce provides basic measures national economy current analysis short-run changes economic situation business outlook ', {'The_Office_of_Business_Economics': 'group-n-1', 'U.S._Department_of_Commerce': 'group-n-1', 'provides': 'provide-v-1', 'basic': 'basic-a-1', 'measures': 'measure-n-1', 'national': 'national-a-5', 'economy': 'economy-n-1', 'current': 'current-a-1', 'analysis': 'analysis-n-1', 'short-run': 'short-run-a-1', 'changes': 'change-n-1', 'economic': 'economic-a-1', 'situation': 'situation-n-1', 'business': 'business-n-3', 'outlook': 'outlook-n-2'}], 1: ['develops analyzes national_income balance_of_international_payments many other business indicators ', {'develops': 'develop-v-7', 'analyzes': 'analyze-v-1', 'national_income': 'national_income-n-1', 'balance_of_international_payments': 'balance_of_international_payments-n-1', 'many': 'many-a-1', 'other': 'other-a-1', 'business': 'business-n-3', 'indicators': 'indi

### Semcor data reading and creating a dictionary for Brown Corpus Category Hobbies. It contains 2962 sentences.

In [78]:
## Semcor data reading and creating a dictionary for Brown Corpus Category News
counter=0
for file in arr_sem_cor_file_hobbies:
    with open(file) as fp:
        soup = BeautifulSoup(fp, "html.parser")

    tags = soup.find_all("s")
    for i in tags:
        tag=i.find_all("wf", {"cmd":"done"})
        sen1=""
        dict3={}
        for j in tag:
            if type(j.get("lemma"))==str:
                flag=True
                if (not (j.get("ot"))):
                    sen1+=j.text
                    sen1+=" "
                    key=j.text
                    if j.get("pos") in dict_for_pos_tags.keys():
                        pos=dict_for_pos_tags[j.get("pos")]
                    else:
                        print("Pos not available")
                    value=j.get("lemma")+"-"+pos+"-"+j.get("wnsn")
                    dict3[key]=value
            else:
                flag=False
        
        if flag:
            key=counter
            value=sen1
            dict_for_hobbies[key]=[value,dict3]
            counter+=1
            
print(dict_for_hobbies)  

{0: ["Too often beginning bodybuilder do training secretly parents n't want sonny-boy lift all barbell things stunt growth childish taunts schoolmates Mr._America muscles has time ", {'Too': 'too-r-1', 'often': 'often-r-1', 'beginning': 'beginning-a-1', 'bodybuilder': 'bodybuilder-n-1', 'do': 'do-v-1', 'training': 'training-n-1', 'secretly': 'secretly-r-1', 'parents': 'parent-n-1', "n't": "n't-r-1", 'want': 'want-v-1', 'sonny-boy': 'sonny_boy-n-1', 'lift': 'lift-v-8', 'all': 'all-a-1', 'barbell': 'barbell-n-1', 'things': 'thing-n-3', 'stunt': 'stunt-v-1', 'growth': 'growth-n-1', 'childish': 'childish-a-1', 'taunts': 'taunt-n-1', 'schoolmates': 'schoolmate-n-1', 'Mr._America': 'person-n-1', 'muscles': 'muscle-n-1', 'has': 'have-v-2', 'time': 'time-n-2'}], 1: ['After_all guy have a_little ego ', {'After_all': 'after_all-r-1', 'guy': 'guy-n-1', 'have': 'have-v-3', 'a_little': 'a_little-r-1', 'ego': 'ego-n-1'}], 2: ['Therefore genuine pleasure tell entirely happy bodybuilder never train in

### Semcor data reading and creating a dictionary for Brown Corpus Category Adventure. It contains 2591 sentences.

In [80]:
## Semcor data reading and creating a dictionary for Brown Corpus Category News
counter=0
for file in arr_sem_cor_file_adventure:
    with open(file) as fp:
        soup = BeautifulSoup(fp, "html.parser")

    tags = soup.find_all("s")
    for i in tags:
        tag=i.find_all("wf", {"cmd":"done"})
        sen1=""
        dict3={}
        for j in tag:
            if type(j.get("lemma"))==str:
                flag=True
                if (not (j.get("ot"))):
                    sen1+=j.text
                    sen1+=" "
                    key=j.text
                    if j.get("pos") in dict_for_pos_tags.keys():
                        pos=dict_for_pos_tags[j.get("pos")]
                    else:
                        print("Pos not available")
                    value=j.get("lemma")+"-"+pos+"-"+j.get("wnsn")
                    dict3[key]=value
            else:
                flag=False
        
        if flag:
            key=counter
            value=sen1
            dict_for_adventure[key]=[value,dict3]
            counter+=1
            
print(dict_for_adventure)  

{0: ['carrying quirt started raise then let fall again dangle wrist ', {'carrying': 'carry-v-2', 'quirt': 'quirt-n-1', 'started': 'start-v-1', 'raise': 'raise-v-2', 'then': 'then-r-1', 'let': 'let-v-1', 'fall': 'fall-v-2', 'again': 'again-r-1', 'dangle': 'dangle-v-1', 'wrist': 'wrist-n-1'}], 1: ['saw fire said speaking slowly making effort control anger ', {'saw': 'see-v-1', 'fire': 'fire-n-1', 'said': 'say-v-1', 'speaking': 'speak-v-1', 'slowly': 'slowly-r-1', 'making': 'make-v-1', 'effort': 'effort-n-1', 'control': 'control-v-2', 'anger': 'anger-n-1'}], 2: ['burn_down whole mountainside fire size ', {'burn_down': 'burn_down-v-1', 'whole': 'whole-a-1', 'mountainside': 'mountainside-n-1', 'fire': 'fire-n-1', 'size': 'size-n-1'}], 3: ["n't matter_to fool ", {"n't": "n't-r-1", 'matter_to': 'matter_to-v-1', 'fool': 'fool-n-1'}], 4: ['All_right Wilson said quickly ', {'All_right': 'all_right-r-1', 'Wilson': 'person-n-1', 'said': 'say-v-1', 'quickly': 'quickly-r-1'}], 5: ['fire too big ', {

### For Brown Corpus Category Government

In [214]:
final_arr_brown_government=[]
for i in dict_for_government.keys():
    final_arr_brown_government.append(calculate_meaning(dict_for_government[i][0],dict_for_government[i][1]))

flat_list = [item for sublist in final_arr_brown_government for item in sublist]
accuracy_final_brown_government=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy for Brown Corpus Government category is: ",accuracy_final_brown_government*100)

Length of final words 11712
Final Accuracy for Brown Corpus Government category is:  42.46926229508197


### For Brown Corpus Category Hobbies

In [215]:
final_arr_brown_hobbies=[]
for i in dict_for_hobbies.keys():
    final_arr_brown_hobbies.append(calculate_meaning(dict_for_hobbies[i][0],dict_for_hobbies[i][1]))

flat_list = [item for sublist in final_arr_brown_hobbies for item in sublist]
accuracy_final_brown_hobbies=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy for Brown Corpus Hobbies category is: ",accuracy_final_brown_hobbies*100)

Length of final words 17364
Final Accuracy for Brown Corpus Hobbies category is:  40.43999078553328


### For Brown Corpus Category Adventure

In [216]:
final_arr_brown_adventure=[]
for i in dict_for_adventure.keys():
    final_arr_brown_adventure.append(calculate_meaning(dict_for_adventure[i][0],dict_for_adventure[i][1]))

flat_list = [item for sublist in final_arr_brown_adventure for item in sublist]
accuracy_final_brown_adventure=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy for Brown Corpus Adventure category is: ",accuracy_final_brown_adventure*100)

Length of final words 11096
Final Accuracy for Brown Corpus Adventure category is:  41.53749098774333


### Extension 3: Train your own word embeddings for this task, possibly initializing the embeddings with pre-trained embeddings.

In [209]:
cores = multiprocessing.cpu_count()
model_for_embedding = Word2Vec(min_count=1,
                  window=3,
                  vector_size=300,  # this parameter is called 'vector_size' in more recent versions of gensim
                  workers=cores,
                  sg=1
                 )

### All sentences in tokenized form are taken from Semcor, SenseEval2 and SenseEval3 to build vocabulary for training own embedding

In [85]:
##Building Vocab for training our own model:
dict_vocab_list=[dict2,dict_senseval2,dict_senseval3]
sentences_array=[]
for dict_inter in dict_vocab_list:
    for j in dict_inter.keys():
        sentences_array.append(word_tokenize(dict_inter[j][0]))       

In [210]:
# defining the vocabulary based on our data
model_for_embedding.build_vocab(sentences_array, progress_per=10000)

# training
model_for_embedding.train(sentences_array, total_examples=model_for_embedding.corpus_count, epochs=10, report_delay=1)

(2137622, 2242950)

In [211]:
def find_gloss_embedding_new(sense):
    
    
    """
    Input: Sense of a word
    Output: Array of word embeddings of all words in the gloss. Here newly trained embedding is considered.
    
    """
    
    gloss = set(word_tokenize(sense.definition()))           
    gloss = gloss.difference( functionwords )
    examples_arr=sense.examples()
    new_token_array=[word_tokenize(i) for i in examples_arr]
    examples_new_token_array = [item for sublist in new_token_array for item in sublist]
    examples_new_token_array=set(examples_new_token_array)
    gloss = gloss.union( examples_new_token_array )
    gloss = gloss.difference( functionwords )
    arr=[]
    
    for i in gloss:
        if i in model_for_embedding.wv.key_to_index:
            word_embedding_newvocab=model_for_embedding.wv[i]
            arr.append(word_embedding_newvocab)
    return arr

### train_embedding() function uses newly trained model "model_for_embedding" for finding word embeddings.

In [212]:
##Training my own embedding based on our vocab building
def train_embedding(sen1,dict_for_words): 
    acc_arr=[]
    set_of_words=set(word_tokenize(sen1))  
    ## For sorting the words based on number of synsets each word has before passing it to program
    sorted_list=[]
    for i in set_of_words:
        if len(wordnet.synsets(i))>0:
            sorted_list.append([i,len(wordnet.synsets(i))])

    sorted_list.sort(key = lambda x: x[1])
    

    dict_for_sense_embedding={}
    for word_list in sorted_list:
        word=word_list[0]
        final_score_arr=[]

        for sense in wordnet.synsets(word): 
            
            arr=find_gloss_embedding_new(sense)
            
            gloss_embedding_sense=np.mean(arr, axis=0) #Gloss embedding for 1 sense of a word
            
            gloss_embedding_sense=gloss_embedding_sense.tolist()

            sentence1=set(word_tokenize(sen1))
            sentence1 = sentence1.difference( functionwords )


            arr2=[x for x in sentence1 if x not in word]
            ## This is for Context embedding
            arr3=[]
            for i in arr2:
                if i in dict_for_sense_embedding.keys():
                    #### This is sense embedding for disambiguated word
                    arr3=find_gloss_embedding_new(dict_for_sense_embedding[i])
                    
                    
                else:
                    if i in model_for_embedding.wv.key_to_index:  
                        word_embedding_newvocab=model_for_embedding.wv[i]
                        arr3.append(word_embedding_newvocab)

            context_embedding_sentence=np.mean(arr3, axis=0) #Context embedding for entire sentence removing the word
            context_embedding_sentence=context_embedding_sentence.tolist()
            score_first_part= spatial.distance.cosine(gloss_embedding_sense, context_embedding_sentence)


            offset=sense.offset()

            
            if word in dict_for_words.keys():
                lemma=dict_for_words[word].split("-")[0]
                pos=dict_for_words[word].split("-")[1]
                wnsn=dict_for_words[word].split("-")[2]
                format_word=lemma+"-wn-2.1-"+str(offset).zfill(8)+"-"+pos


                if format_word in dict1.keys():
                    lexeme_vector=dict1[format_word]
                    score_second_part= spatial.distance.cosine(lexeme_vector, context_embedding_sentence)
                    final_score=score_first_part+score_second_part
                else:
                    final_score=score_first_part
            else:
                final_score=score_first_part
            
            final_score_arr.append(final_score)

        if len(wordnet.synsets(word))!=0:
            max_score=np.argmax(final_score_arr)
            key=word
            value=wordnet.synsets(word)[max_score]
            dict_for_sense_embedding[key]=value
            
            predicted_synset_definition=wordnet.synsets(word)[max_score].definition()
            predicted_synset=wordnet.synsets(word)[max_score].name()

            if word in dict_for_words.keys():
                if (word not in dict_for_words.keys() and word+'.' in dict_for_words.keys()): 
                    word=word+'.'
                lemma=dict_for_words[word].split("-")[0]
                pos=dict_for_words[word].split("-")[1]
                wnsn=dict_for_words[word].split("-")[2]
                true_synset=lemma+'.'+pos+'.0'+wnsn
            
                acc=find_accuracy(predicted_synset,true_synset)
                acc_arr.append(acc)
            
    return acc_arr

### Evaluation on SemCor

In [213]:
final_arr_semcor_embedding=[]
for i in random_list:
    final_arr_semcor_embedding.append(train_embedding(dict2[i][0],dict2[i][1]))

flat_list = [item for sublist in final_arr_semcor_embedding for item in sublist]
accuracy_final_semcor_embedding=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy for Semcor by training own embedding is: ",accuracy_final_semcor_embedding*100)

Length of final words 29189
Final Accuracy for Semcor by training own embedding is:  41.15591489944842


### Evaluation on SenseEval2

In [167]:
##Senseval 2 Computation
final_arr_senseval2_embedding=[]
for i in dict_senseval2.keys():
    final_arr_senseval2_embedding.append(train_embedding(dict_senseval2[i][0],dict_senseval2[i][1]))

flat_list = [item for sublist in final_arr_senseval2_embedding for item in sublist]
accuracy_final_senseval2_embedding=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy for Senseval2 by training own embedding is",accuracy_final_senseval2_embedding*100)

Length of final words 2373
Final Accuracy for Senseval2 by training own embedding is 40.62368310155921


### Evaluation on SenseEval3

In [168]:
##Senseval3 Computation
final_arr_senseval3_embedding=[]
for i in dict_senseval3.keys():
    final_arr_senseval3_embedding.append(train_embedding(dict_senseval3[i][0],dict_senseval3[i][1]))

flat_list = [item for sublist in final_arr_senseval3_embedding for item in sublist]
accuracy_final_senseval3_embedding=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy is for Senseval3 by training own embedding is",accuracy_final_senseval3_embedding*100)

Length of final words 1941
Final Accuracy is for Senseval3 by training own embedding is 36.26996393611541


### Extension 4: Use several pre-trained embeddings or train embeddings with various parameter settings (you probably need to make big changes so you actually get signicantly different results for WSD) and study the influence of the used embeddings on the disambiguation task.

In [199]:
cores = multiprocessing.cpu_count()
model_embedding_p1 = Word2Vec(sentences_array,min_count=5,
                  window=5,
                  vector_size=300,  # this parameter is called 'vector_size' in more recent versions of gensim
                  workers=cores,
                  sg=1
                 )

In [200]:
def find_gloss_embedding_p1(sense):
    gloss = set(word_tokenize(sense.definition()))           
    gloss = gloss.difference( functionwords )
    examples_arr=sense.examples()
    new_token_array=[word_tokenize(i) for i in examples_arr]
    examples_new_token_array = [item for sublist in new_token_array for item in sublist]
    examples_new_token_array=set(examples_new_token_array)
    gloss = gloss.union( examples_new_token_array )
    gloss = gloss.difference( functionwords )
    arr=[]
    for i in gloss:
        if i in model_embedding_p1.wv.key_to_index:
            word_embedding_p1=model_embedding_p1.wv[i]
            arr.append(word_embedding_p1)
    return arr

### train_embedding_p1() function uses model "model_embedding_p1" for computing word embedding.

In [201]:
##Training based on our vocab building
def train_embedding_p1(sen1,dict_for_words): 
    acc_arr=[]
    set_of_words=set(word_tokenize(sen1))  
    ## For sorting the words based on number of synsets each word has before passing it to program
    sorted_list=[]
    for i in set_of_words:
        if len(wordnet.synsets(i))>0:
            sorted_list.append([i,len(wordnet.synsets(i))])

    sorted_list.sort(key = lambda x: x[1])
    

    dict_for_sense_embedding={}
    for word_list in sorted_list:
        word=word_list[0]
        
        final_score_arr=[]

        for sense in wordnet.synsets(word): 
            
            arr=find_gloss_embedding_p1(sense)
 
            gloss_embedding_sense=np.mean(arr, axis=0) #Gloss embedding for 1 sense of a word
            
            gloss_embedding_sense=gloss_embedding_sense.tolist()
            sentence1=set(word_tokenize(sen1))
            sentence1 = sentence1.difference( functionwords )


            arr2=[x for x in sentence1 if x not in word]
            ## This is for Context embedding
            arr3=[]
            for i in arr2:
                if i in dict_for_sense_embedding.keys():
                    #### This is sense embedding for disambiguated word
                    arr3=find_gloss_embedding_p1(dict_for_sense_embedding[i])
                    
                else:
                    if i in model_embedding_p1.wv.key_to_index:  
                        word_embedding_p1=model_embedding_p1.wv[i]
                        arr3.append(word_embedding_p1)

            context_embedding_sentence=np.mean(arr3, axis=0) #Context embedding for entire sentence removing the word
            context_embedding_sentence=context_embedding_sentence.tolist()
            score_first_part= spatial.distance.cosine(gloss_embedding_sense, context_embedding_sentence)
            

            offset=sense.offset()
            
            if word in dict_for_words.keys():
                lemma=dict_for_words[word].split("-")[0]
                pos=dict_for_words[word].split("-")[1]
                wnsn=dict_for_words[word].split("-")[2]
                format_word=lemma+"-wn-2.1-"+str(offset).zfill(8)+"-"+pos

                if format_word in dict1.keys():
                    lexeme_vector=dict1[format_word]
                    score_second_part= spatial.distance.cosine(lexeme_vector, context_embedding_sentence)
                    final_score=score_first_part+score_second_part
                else:
                    final_score=score_first_part
            else:
                final_score=score_first_part
            
            final_score_arr.append(final_score)

        if len(wordnet.synsets(word))!=0:
            max_score=np.argmax(final_score_arr)

            key=word
            value=wordnet.synsets(word)[max_score]
            dict_for_sense_embedding[key]=value
            
            predicted_synset_definition=wordnet.synsets(word)[max_score].definition()
            predicted_synset=wordnet.synsets(word)[max_score].name()
            
            if word in dict_for_words.keys():
                if (word not in dict_for_words.keys() and word+'.' in dict_for_words.keys()): 
                    word=word+'.'
                lemma=dict_for_words[word].split("-")[0]
                pos=dict_for_words[word].split("-")[1]
                wnsn=dict_for_words[word].split("-")[2]
                true_synset=lemma+'.'+pos+'.0'+wnsn
            
                acc=find_accuracy(predicted_synset,true_synset)
                acc_arr.append(acc)

    return acc_arr

### Evaluation of SemCor with parameter changes in training own embedding in min_count and window.

In [202]:
final_arr_semcor_embed_p1=[]
for i in random_list:
    final_arr_semcor_embed_p1.append(train_embedding_p1(dict2[i][0],dict2[i][1]))

flat_list = [item for sublist in final_arr_semcor_embed_p1 for item in sublist]
accuracy_final_semcor_embed_p1=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy for Semcor by training own embedding with parameter change is: ",accuracy_final_semcor_embed_p1*100)

Length of final words 29189
Final Accuracy for Semcor by training own embedding with parameter change is:  41.22443386207133


### Evaluation of SenseEval2 with parameter changes in training own embedding in min_count and window.

In [172]:
##Senseval 2 Computation
final_arr_senseval2_embed_p1=[]
for i in dict_senseval2.keys():
    final_arr_senseval2_embed_p1.append(train_embedding_p1(dict_senseval2[i][0],dict_senseval2[i][1]))

flat_list = [item for sublist in final_arr_senseval2_embed_p1 for item in sublist]
accuracy_final_senseval2_embed_p1=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy for Senseval2 by training own embedding with parameter change is",accuracy_final_senseval2_embed_p1*100)

Length of final words 2373
Final Accuracy for Senseval2 by training own embedding with parameter change is 41.677201854193


### Evaluation of SenseEval3 with parameter changes in training own embedding in min_count and window.

In [173]:
##Senseval3 Computation
final_arr_senseval3_embed_p1=[]
for i in dict_senseval3.keys():
    final_arr_senseval3_embed_p1.append(train_embedding_p1(dict_senseval3[i][0],dict_senseval3[i][1]))

flat_list = [item for sublist in final_arr_senseval3_embed_p1 for item in sublist]
accuracy_final_senseval3_embed_p1=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy is for Senseval3 by training own embedding with parameter change is",accuracy_final_senseval3_embed_p1*100)

Length of final words 1941
Final Accuracy is for Senseval3 by training own embedding with parameter change is 37.14580113343637


### After changing Window Size

In [185]:
cores = multiprocessing.cpu_count()
model_embedding_p2 = Word2Vec(sentences_array,min_count=1,window=30,vector_size=300,workers=cores,sg=1)

In [186]:
def find_gloss_embedding_p2(sense):
    
    gloss = set(word_tokenize(sense.definition()))           
    gloss = gloss.difference( functionwords )
    examples_arr=sense.examples()
    new_token_array=[word_tokenize(i) for i in examples_arr]
    examples_new_token_array = [item for sublist in new_token_array for item in sublist]
    examples_new_token_array=set(examples_new_token_array)
    gloss = gloss.union( examples_new_token_array )
    gloss = gloss.difference( functionwords )
    arr=[]
    for i in gloss:
        if i in model_embedding_p2.wv.key_to_index:
            word_embedding_p2=model_embedding_p2.wv[i]
            arr.append(word_embedding_p2)
    return arr

### train_embedding_p2() function uses model "model_embedding_p2" for computing word embedding.

In [197]:
##Training based on our vocab building
def train_embedding_p2(sen1,dict_for_words): 
    acc_arr=[]
    set_of_words=set(word_tokenize(sen1))  
    ## For sorting the words based on number of synsets each word has before passing it to program
    sorted_list=[]
    for i in set_of_words:
        if len(wordnet.synsets(i))>0:
            sorted_list.append([i,len(wordnet.synsets(i))])

    sorted_list.sort(key = lambda x: x[1])
    
    dict_for_sense_embedding={}
    for word_list in sorted_list:
        
        word=word_list[0]

        final_score_arr=[]
        
            
        for sense in wordnet.synsets(word): 
            
            arr=find_gloss_embedding_p2(sense)
            
            
            gloss_embedding_sense=np.mean(arr, axis=0) #Gloss embedding for 1 sense of a word
            
            gloss_embedding_sense=gloss_embedding_sense.tolist()

            sentence1=set(word_tokenize(sen1))
            sentence1 = sentence1.difference( functionwords )


            arr2=[x for x in sentence1 if x not in word]
            ## This is for Context embedding
            arr3=[]
            for i in arr2:
                if i in dict_for_sense_embedding.keys():
                    #### This is sense embedding for disambiguated word
                    arr3=find_gloss_embedding_p2(dict_for_sense_embedding[i])
                    
                    
                else:
                    if i in model_embedding_p2.wv.key_to_index:  
                        word_embedding_p2=model_embedding_p2.wv[i]
                        arr3.append(word_embedding_p2)

            context_embedding_sentence=np.mean(arr3, axis=0) #Context embedding for entire sentence removing the word
            context_embedding_sentence=context_embedding_sentence.tolist()
            score_first_part= spatial.distance.cosine(gloss_embedding_sense, context_embedding_sentence)
            
            offset=sense.offset()
            
            if word in dict_for_words.keys():
                lemma=dict_for_words[word].split("-")[0]
                pos=dict_for_words[word].split("-")[1]
                wnsn=dict_for_words[word].split("-")[2]
                format_word=lemma+"-wn-2.1-"+str(offset).zfill(8)+"-"+pos
                if format_word in dict1.keys():
                    lexeme_vector=dict1[format_word]
                    score_second_part= spatial.distance.cosine(lexeme_vector, context_embedding_sentence)
                    final_score=score_first_part+score_second_part
                else:
                    final_score=score_first_part
            else:
                final_score=score_first_part
            
            final_score_arr.append(final_score)

        if len(wordnet.synsets(word))!=0:
            max_score=np.argmax(final_score_arr)
            key=word
            value=wordnet.synsets(word)[max_score]
            dict_for_sense_embedding[key]=value
            
            predicted_synset_definition=wordnet.synsets(word)[max_score].definition()
            predicted_synset=wordnet.synsets(word)[max_score].name()
            
            if word in dict_for_words.keys():
#                 if (word not in dict_for_words.keys() and word+'.' in dict_for_words.keys()): 
#                     word=word+'.'
                lemma=dict_for_words[word].split("-")[0]
                pos=dict_for_words[word].split("-")[1]
                wnsn=dict_for_words[word].split("-")[2]
                true_synset=lemma+'.'+pos+'.0'+wnsn

                acc=find_accuracy(predicted_synset,true_synset)
                acc_arr.append(acc)

    return acc_arr

In [188]:
def find_accuracy(predicted_synset,true_synset):
    
    predicted_synset_arr=predicted_synset.split(".")
    true_synset_arr=true_synset.split(".")
    if ';' in true_synset_arr[2]:
        true_synset_arr_1=true_synset_arr[2].split(";")
        true_synset_arr_1=[i.zfill(2) for i in true_synset_arr_1]
    else:
        true_synset_arr_1=true_synset_arr[2]
    if predicted_synset_arr[1]=='s':
        predicted_synset_arr[1]='a'
    if (predicted_synset_arr[1]==true_synset_arr[1] and predicted_synset_arr[2] in true_synset_arr_1):
        return 1
    else:
        return 0

### Evaluation of SemCor with parameter changes in training own embedding in min_count and window.

In [198]:
final_arr_semcor_embed_p2=[]
for i in random_list:
    final_arr_semcor_embed_p2.append(train_embedding_p2(dict2[i][0],dict2[i][1]))

flat_list = [item for sublist in final_arr_semcor_embed_p2 for item in sublist]
accuracy_final_semcor_embed_p2=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy for Semcor by training own embedding with parameter change is: ",accuracy_final_semcor_embed_p2*100)

Length of final words 29189
Final Accuracy for Semcor by training own embedding with parameter change is:  40.68998595361266


### Evaluation of SenseEval2 with parameter changes in training own embedding in min_count and window.

In [189]:
##Senseval 2 Computation
final_arr_senseval2_embed_p2=[]
for i in dict_senseval2.keys():
    final_arr_senseval2_embed_p2.append(train_embedding_p2(dict_senseval2[i][0],dict_senseval2[i][1]))

flat_list = [item for sublist in final_arr_senseval2_embed_p2 for item in sublist]
accuracy_final_senseval2_embed_p2=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy for Senseval2 by training own embedding with parameter change is",accuracy_final_senseval2_embed_p2*100)

Length of final words 2373
Final Accuracy for Senseval2 by training own embedding with parameter change is 42.05646860514117


### Evaluation of SenseEval3 with parameter changes in training own embedding in min_count and window.

In [190]:
##Senseval3 Computation
final_arr_senseval3_embed_p2=[]
for i in dict_senseval3.keys():
    final_arr_senseval3_embed_p2.append(train_embedding_p2(dict_senseval3[i][0],dict_senseval3[i][1]))

flat_list = [item for sublist in final_arr_senseval3_embed_p2 for item in sublist]
accuracy_final_senseval3_embed_p2=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy is for Senseval3 by training own embedding with parameter change is",accuracy_final_senseval3_embed_p2*100)

Length of final words 1941
Final Accuracy is for Senseval3 by training own embedding with parameter change is 36.21844410097887


### Extension 5: Extend the word embedding model to also use character-based representations, e.g. fastText.

In [191]:
# Defining values for parameters
embedding_size = 300
window_size = 5
min_word = 1
down_sampling = 1e-2
fast_text = FastText(sentences_array,
                     vector_size=embedding_size,
                     window=window_size,
                     min_count=min_word,
                     sample=down_sampling,
                     workers = 8,
                     sg=1,
                     )

In [192]:
def find_gloss_embedding_fasttext(sense):
    gloss = set(word_tokenize(sense.definition()))           
    gloss = gloss.difference( functionwords )
    examples_arr=sense.examples()
    new_token_array=[word_tokenize(i) for i in examples_arr]
    examples_new_token_array = [item for sublist in new_token_array for item in sublist]
    examples_new_token_array=set(examples_new_token_array)
    gloss = gloss.union( examples_new_token_array )
    gloss = gloss.difference( functionwords )
    arr=[]
    for i in gloss:
        if i in fast_text.wv.key_to_index:
            word_embedding_fasttext=fast_text.wv[i]
            arr.append(word_embedding_fasttext)
    return arr

### fasttext_embedding() function uses FastText model to compute word embeddings and is trained on our data. This model is imported from gensim.models.fasttext. It uses character based representation (using sub-words or n-grams). The parameters selected for training are mentioned above.

In [193]:
def fasttext_embedding(sen1,dict_for_words): 
    acc_arr=[]
    set_of_words=set(word_tokenize(sen1))  
    ## For sorting the words based on number of synsets each word has before passing it to program
    sorted_list=[]
    for i in set_of_words:
        if len(wordnet.synsets(i))>0:
            sorted_list.append([i,len(wordnet.synsets(i))])

    sorted_list.sort(key = lambda x: x[1])
    

    dict_for_sense_embedding={}
    for word_list in sorted_list:
        word=word_list[0]
        
        final_score_arr=[]
            
        for sense in wordnet.synsets(word): 
            arr=find_gloss_embedding_fasttext(sense)
            
            
            gloss_embedding_sense=np.mean(arr, axis=0) #Gloss embedding for 1 sense of a word
            gloss_embedding_sense=gloss_embedding_sense.tolist()
            sentence1=set(word_tokenize(sen1))
            sentence1 = sentence1.difference( functionwords )


            arr2=[x for x in sentence1 if x not in word]
            ## This is for Context embedding
            arr3=[]
            for i in arr2:
                if i in dict_for_sense_embedding.keys():
                    #### This is sense embedding for disambiguated word
                    arr3=find_gloss_embedding_fasttext(dict_for_sense_embedding[i])
                    
                else:
                    if i in fast_text.wv.key_to_index:  
                        word_embedding_fasttext=fast_text.wv[i]
                        arr3.append(word_embedding_fasttext)

            context_embedding_sentence=np.mean(arr3, axis=0) #Context embedding for entire sentence removing the word
            context_embedding_sentence=context_embedding_sentence.tolist()
            score_first_part= spatial.distance.cosine(gloss_embedding_sense, context_embedding_sentence)

            offset=sense.offset()

            if word in dict_for_words.keys():
                lemma=dict_for_words[word].split("-")[0]
                pos=dict_for_words[word].split("-")[1]
                wnsn=dict_for_words[word].split("-")[2]
                format_word=lemma+"-wn-2.1-"+str(offset).zfill(8)+"-"+pos


                if format_word in dict1.keys():
                    lexeme_vector=dict1[format_word]
                    score_second_part= spatial.distance.cosine(lexeme_vector, context_embedding_sentence)
                    final_score=score_first_part+score_second_part
                else:
                    final_score=score_first_part
            else:
                final_score=score_first_part
            
            final_score_arr.append(final_score)
        if len(wordnet.synsets(word))!=0:
            max_score=np.argmax(final_score_arr)
            key=word
            value=wordnet.synsets(word)[max_score]
            dict_for_sense_embedding[key]=value
            
            predicted_synset_definition=wordnet.synsets(word)[max_score].definition()
            predicted_synset=wordnet.synsets(word)[max_score].name()
            
            if word in dict_for_words.keys():
                if (word not in dict_for_words.keys() and word+'.' in dict_for_words.keys()): 
                    word=word+'.'
                lemma=dict_for_words[word].split("-")[0]
                pos=dict_for_words[word].split("-")[1]
                wnsn=dict_for_words[word].split("-")[2]
                true_synset=lemma+'.'+pos+'.0'+wnsn
            
                acc=find_accuracy(predicted_synset,true_synset)
                acc_arr.append(acc)
            
    return acc_arr

### Evaluation of SemCor using FastText Embeddings

In [196]:
final_arr_semcor_fasttext=[]
for i in random_list:
    final_arr_semcor_fasttext.append(fasttext_embedding(dict2[i][0],dict2[i][1]))

flat_list = [item for sublist in final_arr_semcor_fasttext for item in sublist]
accuracy_final_semcor_fasttext=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy for Semcor by using FastText is: ",accuracy_final_semcor_fasttext*100)

Length of final words 29189
Final Accuracy for Semcor by using FastText is:  40.77220870876015


### Evaluation of SenseEval2 using FastText Embeddings

In [194]:
##Senseval 2 Computation
final_arr_senseval2_fasttext=[]
for i in dict_senseval2.keys():
    final_arr_senseval2_fasttext.append(fasttext_embedding(dict_senseval2[i][0],dict_senseval2[i][1]))

flat_list = [item for sublist in final_arr_senseval2_fasttext for item in sublist]
accuracy_final_senseval2_fasttext=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy for Senseval2 by using FastText is",accuracy_final_senseval2_fasttext*100)

Length of final words 2373
Final Accuracy for Senseval2 by using FastText is 43.48925410872314


### Evaluation of SenseEval3 using FastText Embeddings

In [195]:
##Senseval3 Computation
final_arr_senseval3_fasttext=[]
for i in dict_senseval3.keys():
    final_arr_senseval3_fasttext.append(fasttext_embedding(dict_senseval3[i][0],dict_senseval3[i][1]))

flat_list = [item for sublist in final_arr_senseval3_fasttext for item in sublist]
accuracy_final_senseval3_fasttext=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy for Senseval3 by using FastText is",accuracy_final_senseval3_fasttext*100)

Length of final words 1941
Final Accuracy for Senseval3 by using FastText is 37.09428129829985


### Extension 6: Use transformers and sentence embeddings to compare a sentence and a gloss. E.g. you could use the SBERT pre-trained models. Use a part of the data to tune the transformer and classication model

In [217]:
model_sbert = SentenceTransformer('all-MiniLM-L6-v2')

### Here pre-trained model 'all-MiniLM-L6-v2' is used. SBERT is used to provide embedding for entire sentence. 

In [219]:
def calculate_meaning_sbert(sen1,dict_for_words): 
    acc_arr=[]
    set_of_words=set(word_tokenize(sen1))  
    ## For sorting the words based on number of synsets each word has before passing it to program
    sorted_list=[]
    for i in set_of_words:
        if len(wordnet.synsets(i))>0:
            sorted_list.append([i,len(wordnet.synsets(i))])

    sorted_list.sort(key = lambda x: x[1])
    
#     for word in set_of_words:
    dict_for_sense_embedding={}
    for word_list in sorted_list:
        word=word_list[0]
        
        final_score_arr=[]

        for sense in wordnet.synsets(word): 

            arr=[]
            gloss=sense.definition()
            ## Here find sentence encoding for sense definition. Do not remove function words as encoding is for an entire entence.
            embeddings_def = model_sbert.encode(gloss)
            arr.append(embeddings_def)

            
            ### Here is the part where union from examples is being carried out
            
            examples_arr=sense.examples()
            
            for sentence_example in examples_arr:
                embeddings_example = model_sbert.encode(sentence_example)
                arr.append(embeddings_example)
            
            
            gloss_embedding_sense=np.mean(arr, axis=0) #Gloss embedding for 1 sense of a word
            
            gloss_embedding_sense=gloss_embedding_sense.tolist()

            sentence_for_context=set(word_tokenize(sen1))
            sentence_for_context = sentence_for_context.difference( functionwords )
            sentence1 = sen1.replace(word, '')
            embeddings_context = model_sbert.encode(sentence1)

            arr2=[x for x in sentence_for_context if x not in word]
            ## This is for Context embedding
            arr3=[]
            for i in arr2:
                if i in dict_for_sense_embedding.keys():
                    #### This is sense embedding for disambiguated word
                    arr3=find_gloss_embedding(dict_for_sense_embedding[i])
                    
                    
                else:
                    if i in goog_model.key_to_index:  
                        word_embedding=goog_model[i]
                        arr3.append(word_embedding)

            context_embedding_sentence=np.mean(arr3, axis=0) #Context embedding for entire sentence removing the word
            
            context_embedding_sentence=context_embedding_sentence.tolist()
            
            context_embedding_sentence_sbert=embeddings_context
            context_embedding_sentence_sbert=context_embedding_sentence_sbert.tolist()
            if (len(gloss)==0):
                score_first_part= 0
            else:
                score_first_part= spatial.distance.cosine(gloss_embedding_sense, context_embedding_sentence_sbert) ## 384 dimensional vector

            offset=sense.offset()

            
            if word in dict_for_words.keys():
                lemma=dict_for_words[word].split("-")[0]
                pos=dict_for_words[word].split("-")[1]
                wnsn=dict_for_words[word].split("-")[2]
                format_word=lemma+"-wn-2.1-"+str(offset).zfill(8)+"-"+pos


                if format_word in dict1.keys():
                    lexeme_vector=dict1[format_word]
                    score_second_part= spatial.distance.cosine(lexeme_vector, context_embedding_sentence) ## 300 dimensional vector
                    final_score=score_first_part+score_second_part
                else:
                    final_score=score_first_part
            else:
                final_score=score_first_part
            
            final_score_arr.append(final_score)
        if len(wordnet.synsets(word))!=0:
            max_score=np.argmax(final_score_arr)
            key=word
            value=wordnet.synsets(word)[max_score]
            dict_for_sense_embedding[key]=value
            
            predicted_synset_definition=wordnet.synsets(word)[max_score].definition()
            predicted_synset=wordnet.synsets(word)[max_score].name()
            
            if word in dict_for_words.keys():
                if (word not in dict_for_words.keys() and word+'.' in dict_for_words.keys()): 
                    word=word+'.'
                lemma=dict_for_words[word].split("-")[0]
                pos=dict_for_words[word].split("-")[1]
                wnsn=dict_for_words[word].split("-")[2]
                true_synset=lemma+'.'+pos+'.0'+wnsn
            
                acc=find_accuracy(predicted_synset,true_synset)
                acc_arr.append(acc)
            
    return acc_arr

### Randomly 500 sentences are taken from SemCor and evaluation is carried out on those for SBERT as running on the entire corpus was taking a long amount of time.


In [220]:
## Select sentences
random_no_list=[]
for i in range(0,500):
    x = random.randint(1,34374)
    random_no_list.append(x)
    
print("Length of random numbers",len(random_no_list))

Length of random numbers 500


### Evaluation of SBERT on SemCor

In [221]:
final_arr_semcor_sbert=[]
for i in random_no_list:
    final_arr_semcor_sbert.append(calculate_meaning_sbert(dict2[i][0],dict2[i][1]))

flat_list = [item for sublist in final_arr_semcor_sbert for item in sublist]
accuracy_final_semcor_sbert=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy for Semcor by using Pre-trained SBERT is: ",accuracy_final_semcor_sbert*100)

Length of final words 2959
Final Accuracy for Semcor by using Pre-trained SBERT is:  41.53430212909767


### Evaluation of SBERT on Senseval2

In [222]:
##Senseval 2 Computation
final_arr_sbert_senseval2=[]
for i in dict_senseval2.keys():
    final_arr_sbert_senseval2.append(calculate_meaning_sbert(dict_senseval2[i][0],dict_senseval2[i][1]))


flat_list = [item for sublist in final_arr_sbert_senseval2 for item in sublist]
accuracy_final_sbert_senseval2=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy is for Senseval2 by using Pre-trained SBERT is",accuracy_final_sbert_senseval2*100)

Length of final words 2373
Final Accuracy is for Senseval2 by using Pre-trained SBERT is 38.93805309734513


### Evaluation of SBERT on Senseval3

In [223]:
##Senseval3 Computation
final_arr_senseval3_sbert=[]
for i in dict_senseval3.keys():
    final_arr_senseval3_sbert.append(calculate_meaning_sbert(dict_senseval3[i][0],dict_senseval3[i][1]))

flat_list = [item for sublist in final_arr_senseval3_sbert for item in sublist]
accuracy_final_senseval3_sbert=flat_list.count(1)/len(flat_list)
print("Length of final words",len(flat_list))
print("Final Accuracy for Senseval3 by using Pre-trained SBERT is",accuracy_final_senseval3_sbert*100)

Length of final words 1941
Final Accuracy for Senseval3 by using Pre-trained SBERT is 38.22771767130345
